In [2]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [68]:
# Initialization
## Master problem parameters
c = np.array([100, 150])             # objective coefficients
a1 = np.array([1,1]); rhs1 = 120     # constraint 1, coefficients and right hand side
x_lb = np.array([40, 20])            # lower bounds for x

## Subproblem
rhs = np.array([60, 80])                      # coefficinets of righ hand side
s1 = np.array([500, 100, -24, -28]); p1 = 0.4 # scenario 1 with probability p1 - [d1, d2, q1, q2]
s2 = np.array([300, 300, -28, -32]); p2 = 0.6 # scenario 2 with probability p2 - [d1, d2, q1, q2]

## h,T
h = np.array( [ [0, 0, s1[0], s1[1]] , [0, 0, s2[0], s2[1]] ] )
T = np.array( [ [-60,0] , [0, -80] , [0,0] , [0,0] ])
W = np.array([ [6,10] , [8,5] , [1,0] , [0,1] ])

## Models
master = gp.Model('MasterProblem')             # master 
X = master.addMVar(2, lb=x_lb, name='X')       # x variables
master.addConstr(a1@X <= rhs1, name='Const1')  # Ax <= b
master.setObjective(c@X, GRB.MINIMIZE)         # cx

sub = gp.Model('SubProblem')                            # subproblem
Y = sub.addMVar((2,2), lb=0.0, name='Y')                # we have two scenarios so Y is a 2*2 matrix, y1 and y2 for scenario 1 = Y[0] | y1, y2 for scenario 2 = Y[1]
sub.setObjective(Y[0]@s1[2:]+Y[1]@s2[2:], GRB.MINIMIZE) # E(q*y)

## Algorithm parameters
w = 0                       # w = p*pi*h - p*pi*T@x  
termination = float('-inf') # theta
 

    

# Cpnditional Loop
index = 1
while w > termination:    
    # add optimality cut for indices greater than 0 
    if index!=1:
        if index==2: 
            theta = master.addVar(lb=float('-inf'), name='Theta')   # add theta variable to the master problem
            master.setObjective(c@X + theta, GRB.MINIMIZE)          # update master's objective function
        master.addConstr(E@X + theta >= e, name='OpCut'+str(index)) # add the optimalit cut
        
    # solve the master problem
    master.update()
    master.optimize()
    
    # update termination criterion
    if index>= 2:
        termination = theta.x
        
    # update the subproblem   
    if index==1:   
        WS1 = sub.addConstrs((W[i]@Y[0] <= rhs[i]*X[i].x for i in (0,1)), 'WScenario1')   # 6y1 + 8y2 <= 60x1, 8y1 + 5y2 <= 80x2 for scenario 1
        DS1 = sub.addConstrs((W[i+2]@Y[0] <= s1[i] for i in (0,1)), 'DScenario1')         # y1 <= d1, y2 <= d2 forscenario 1
        WS2 = sub.addConstrs((W[i]@Y[1] <= rhs[i]*X[i].x for i in (0,1)), 'WScenario2')   # 6y1 + 8y2 <= 60x1, 8y1 + 5y2 <= 80x2 for scenario 2
        DS2 = sub.addConstrs((W[i+2]@Y[1] <= s2[i] for i in (0,1)), 'DScenario2')         # y1 <= d1, y2 <= d2 for scenario 2 
    else: 
        for i in (0,1):                  # only update those constraints depend on X
            WS1[i].RHS = rhs[i]*X[i].x   # 6y1 + 8y2 <= 60x1, 8y1 + 5y2 <= 80x2 for scenario 1
            WS2[i].RHS = rhs[i]*X[i].x   # 6y1 + 8y2 <= 60x1, 8y1 + 5y2 <= 80x2 for scenario 2
    
    # solve the subproblem
    sub.update()
    sub.optimize()
    
    # get the dual solutions to the subproblem
    temp = []
    for cons in sub.getConstrs():
        temp.append(cons.Pi)  
    Pi = np.array([ np.array(temp[0:len(W)]) , np.array(temp[len(W):])]) # Pi is the dual solutions to the subproblem. First element is for scenario 1 and the sesond is for scenario 2

    # calculate p*pi*h and p*pi*T
    e = Pi[0]@h[0]*p1 + Pi[1]@h[1]*p2
    E = Pi[0]@T*p1 + Pi[1]@T*p2 
    w = e-E@X.x
    
    # Show the steps
    print('*'*100, '\nIteration:', index, ' theta:', termination, ' w:', w); print('*'*100)
    
    index+=1

    
    
# show optimal solution
print('\n\nOptimal Solution is: x1 = %0.3f'%(X[0].x), ', x2 = %0.3f'%(X[1].x))

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD A8-7100 Radeon R5, 8 Compute Cores 4C+4G, instruction set [SSE2|AVX]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xe8492603
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [2e+01, 4e+01]
  RHS range        [1e+02, 1e+02]
Presolve removed 1 rows and 2 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.000000000e+03
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD A8-7100 Radeon R5, 8 Compute Cores 4C+4G, instruction set [SSE2|AVX]
Thread count: 4 physical cores, 4 logical processors, using up to 4 thread